In [1]:
#convert java to python using gpt4o mini and display and run code in an interface

In [64]:
# imports

import os
import io
import sys
import json
import traceback
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [12]:
load_dotenv()
openai_api_key=os.getenv('OPENAI_API_KEY')
openai=OpenAI()
MODEL="gpt-4o-mini"

In [34]:
system_message="You are a code convertor assistant whose job is to reimplement code written in java to python\
for a Dell ARM64 Snapdragon X Plus Windows laptop. Respond only with python and include the imports and use comments\
and explanantions sparingly. Ensure that the output, logic of both the codes are identical for any given input\
and that the output is produced in the fastest possible time. You must maintain the purpose of the code and not\
change the logic, utility or output. DO NOT include example or sample outputs!"


In [35]:
def user_prompt(java_code):
    user_prompt="Below attached is the java code which you must convert to python code while ensuring that the \
    output, purpose and logic of the program is maintained identical. Respond only with python and use comments \
    sparingly."
    user_prompt+=java_code
    return user_prompt

In [36]:
def messages_for(java_code):
    messages=[
        {"role":"system","content":system_message},
        {"role":"user","content":user_prompt(java_code)}
    ]
    return messages

In [37]:
def write_output(python_code):
    code = python_code.replace("```python","").replace("```","")
    with open("optimized.py", "w") as f:
        f.write(code)

In [48]:

def optimize(java_code):
    stream=openai.chat.completions.create(model=MODEL, messages=messages_for(java_code),stream=True)
    reply=""
    for chunk in stream:
        fragment=chunk.choices[0].delta.content or ""
        reply+=fragment
        print(fragment,end=' ',flush=True)
    write_output(reply)
    return reply

In [51]:
#Sample java code for testing - checking for an armstrong number
java_code="""
import java.util.Scanner;

public class ArmstrongChecker {
    public static void main(String[] args) {
        Scanner scanner = new Scanner(System.in);
        int number = scanner.nextInt();
        int originalNumber = number;
        int numDigits = String.valueOf(number).length();
        int sum = 0;
        
        while (number > 0) {
            int digit = number % 10;
            sum += Math.pow(digit, numDigits);
            number /= 10;
        }
        
        if (sum == originalNumber) {
            System.out.println("Armstrong number");
        } else {
            System.out.println("Not an Armstrong number");
        }
        
        scanner.close();
    }
}
"""

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        java=gr.Textbox(label="Java Code:", lines=25, value="",placeholder="Type your Java code here...")
        python=gr.Textbox(label="Python Code:",lines=25)
    with gr.Row():
        convert=gr.Button("Convert code")
    convert.click(optimize,inputs=[java],outputs=[python])
ui.launch(inbrowser=True)
    

In [78]:
def python_execute(code,user_input):
    try:
        cleaned_code = code.strip()
        cleaned_code = cleaned_code.replace("```python", "").replace("```", "")
        
        
        if not user_input.strip():
            return "Error: No input provided. Please enter a value in the 'Python Input' textbox."
        
        
        input_lines = [line.strip() for line in user_input.splitlines() if line.strip()]
        formatted_input = "\n".join(input_lines) + "\n"  
        
       
        input_buffer = io.StringIO(formatted_input)
        sys.stdin = input_buffer
        
        # Capture stdout
        output = io.StringIO()
        sys.stdout = output
        
        # Execute the code (only once)
        exec(cleaned_code)
        
        # Restore stdout and stdin
        sys.stdout = sys.__stdout__
        sys.stdin = sys.__stdin__
        return output.getvalue() or "No output (code executed successfully)"
    except ValueError as e:
        sys.stdout = sys.__stdout__
        sys.stdin = sys.__stdin__
        return f"Error: Invalid input. Please enter a valid number.\n{str(e)}\n{traceback.format_exc()}"
    except EOFError as e:
        sys.stdout = sys.__stdout__
        sys.stdin = sys.__stdin__
        return f"Error: EOFError - Not enough input provided for the number of input() calls.\n{str(e)}\n{traceback.format_exc()}"
    except Exception as e:
        sys.stdout = sys.__stdout__
        sys.stdin = sys.__stdin__
        return f"Error: {str(e)}\n{traceback.format_exc()}"

In [79]:
css="""
#.java {background-color: #306998;}
.python{background-color: #050}
"""

In [80]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Java to Python")
    with gr.Row():
        java = gr.Textbox(label="Java Code:", lines=20, value="", placeholder="Type your Java code here...")
        python = gr.Textbox(label="Python Code:", lines=20)
    with gr.Row():
        convert = gr.Button("Convert code")
        #java_run = gr.Button("Run Java")  # Not implemented
        python_run = gr.Button("Run Python")
    with gr.Row():
        python_input = gr.Textbox(label="Python Input (one line per input call):", placeholder="Enter input for Python code...")
    with gr.Row():
        #java_out = gr.TextArea(label="Java Results:", elem_classes=["java"])
        python_out = gr.TextArea(label="Python Results:", elem_classes=["python"])

    convert.click(optimize, inputs=[java], outputs=[python])
    python_run.click(python_execute, inputs=[python, python_input], outputs=[python_out])

ui.launch(inbrowser=True)